In [ ]:
%load_ext kedro.extras.extensions.ipython

In [ ]:
%reload_kedro

In [ ]:
catalog.list()

In [ ]:
context.catalog.load('params:image_resizer.output_size')

In [ ]:
context.catalog.load('params:image_resizer.method')

In [ ]:
context.catalog.load('input_images')

In [ ]:
from pathlib import Path

In [ ]:
next(Path('../data/01_raw/images_128_128/').glob('*.jpg'))

In [ ]:
context.catalog.load('resized_images')

In [ ]:
from gid_ml_framework.extras.datasets import DirWithImagesDataSet

### loading

In [ ]:
my_dataset = DirWithImagesDataSet('../data/01_raw/images/010/')

In [ ]:
my_data = my_dataset.load()

In [ ]:
new_dict = dict()

In [ ]:
for i, (key, value) in enumerate(my_data.items()):
    if i < 2:
        print(i)
        print(key)
        new_dict[key] = value

In [ ]:
len(new_dict)

### saving

In [ ]:
saving_dataset = DirWithImagesDataSet('../data/02_intermediate/')

In [ ]:
saving_dataset._save(data=new_dict, folder_name='test_images', img_extension='jpg')

In [ ]:
from PIL import Image

In [ ]:
Image.open('../data/01_raw/images/010/0108775015.jpg').convert('RGB')

In [ ]:
# https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial9/AE_CIFAR10.html#Building-the-autoencoder

In [ ]:
catalog.list()

In [ ]:
my_dict = context.catalog.load('parameters')['input']

In [ ]:
img_path, suffix = my_dict['image_path'], my_dict['suffix']

In [ ]:
img_path, suffix

In [ ]:
import os
from torch import optim, nn, utils, Tensor
import pytorch_lightning as pl
from torch.utils.data import DataLoader, Dataset

In [ ]:
import os
from pathlib import Path
from PIL import Image
from torchvision import transforms

In [ ]:
class HMDataset(Dataset):
    
    def __init__(self, img_dir, transform=None):
        self.img_dir = Path(img_dir)
        self.img_names = self._get_img_names()
        self.transform = transform
        
    def _get_img_names(self):
        return [img_name.name for img_name in self.img_dir.iterdir()]

    def __getitem__(self, idx):
        img_path = self.img_dir / self.img_names[idx]
        img = Image.open(img_path)
        
        if self.transform is not None:
            img = self.transform(img)
        
        article_id = img_path.name
        return img, article_id

    def __len__(self):
        return len(self.img_names)

In [ ]:
dataset = HMDataset('/Users/mmadej/Desktop/Projects/gid-ml-framework/data/01_raw/images_128_128/', transform=transforms.ToTensor())

In [ ]:
dataset

In [ ]:
dataloader = DataLoader(dataset=dataset,
                          batch_size=32,
                          drop_last=True,
                          shuffle=True,
                          num_workers=0)

In [ ]:
class Encoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of input channels of the image. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the first convolutional layers. Deeper layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the encoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.net = nn.Sequential(
            nn.Conv2d(num_input_channels, c_hid, kernel_size=3, padding=1, stride=2), # 32x32 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 16x16 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1, stride=2), # 8x8 => 4x4
            act_fn(),
            nn.Flatten(), # Image grid to single feature vector
            nn.Linear(2*16*c_hid, latent_dim)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Decoder(nn.Module):

    def __init__(self,
                 num_input_channels : int,
                 base_channel_size : int,
                 latent_dim : int,
                 act_fn : object = nn.GELU):
        """
        Inputs:
            - num_input_channels : Number of channels of the image to reconstruct. For CIFAR, this parameter is 3
            - base_channel_size : Number of channels we use in the last convolutional layers. Early layers might use a duplicate of it.
            - latent_dim : Dimensionality of latent representation z
            - act_fn : Activation function used throughout the decoder network
        """
        super().__init__()
        c_hid = base_channel_size
        self.linear = nn.Sequential(
            nn.Linear(latent_dim, 2*16*c_hid),
            act_fn()
        )
        self.net = nn.Sequential(
            nn.ConvTranspose2d(2*c_hid, 2*c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 4x4 => 8x8
            act_fn(),
            nn.Conv2d(2*c_hid, 2*c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(2*c_hid, c_hid, kernel_size=3, output_padding=1, padding=1, stride=2), # 8x8 => 16x16
            act_fn(),
            nn.Conv2d(c_hid, c_hid, kernel_size=3, padding=1),
            act_fn(),
            nn.ConvTranspose2d(c_hid, num_input_channels, kernel_size=3, output_padding=1, padding=1, stride=2), # 16x16 => 32x32
            nn.Tanh() # The input images is scaled between -1 and 1, hence the output has to be bounded as well
        )

    def forward(self, x):
        x = self.linear(x)
        x = x.reshape(x.shape[0], -1, 4, 4)
        x = self.net(x)
        return x

In [ ]:
class LitAutoEncoder(pl.LightningModule):

    def __init__(self,
                 base_channel_size: int,
                 latent_dim: int,
                 encoder_class : object = Encoder,
                 decoder_class : object = Decoder,
                 num_input_channels: int = 3,
                 width: int = 32,
                 height: int = 32):
        super().__init__()
        # Saving hyperparameters of autoencoder
        self.save_hyperparameters()
        # Creating encoder and decoder
        self.encoder = encoder_class(num_input_channels, base_channel_size, latent_dim)
        self.decoder = decoder_class(num_input_channels, base_channel_size, latent_dim)
        # Example input array needed for visualizing the graph of the network
        self.example_input_array = torch.zeros(2, num_input_channels, width, height)

    def forward(self, x):
        """
        The forward function takes in an image and returns the reconstructed image
        """
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

    def _get_reconstruction_loss(self, batch):
        """
        Given a batch of images, this function returns the reconstruction loss (MSE in our case)
        """
        x, _ = batch # We do not need the labels
        x_hat = self.forward(x)
        loss = F.mse_loss(x, x_hat, reduction="none")
        loss = loss.sum(dim=[1,2,3]).mean(dim=[0])
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        # Using a scheduler is optional but can be helpful.
        # The scheduler reduces the LR if the validation performance hasn't improved for the last N epochs
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,
                                                         mode='min',
                                                         factor=0.2,
                                                         patience=20,
                                                         min_lr=5e-5)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def training_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        loss = self._get_reconstruction_loss(batch)
        self.log('test_loss', loss)

In [ ]:
class GenerateCallback(pl.Callback):

    def __init__(self, input_imgs, every_n_epochs=1):
        super().__init__()
        self.input_imgs = input_imgs # Images to reconstruct during training
        self.every_n_epochs = every_n_epochs # Only save those images every N epochs (otherwise tensorboard gets quite large)

    def on_epoch_end(self, trainer, pl_module):
        if trainer.current_epoch % self.every_n_epochs == 0:
            # Reconstruct images
            input_imgs = self.input_imgs.to(pl_module.device)
            with torch.no_grad():
                pl_module.eval()
                reconst_imgs = pl_module(input_imgs)
                pl_module.train()
            # Plot and add to tensorboard
            imgs = torch.stack([input_imgs, reconst_imgs], dim=1).flatten(0,1)
            grid = torchvision.utils.make_grid(imgs, nrow=2, normalize=True, range=(-1,1))
            trainer.logger.experiment.add_image("Reconstructions", grid, global_step=trainer.global_step)

In [ ]:
def train_cifar(latent_dim):
    # Create a PyTorch Lightning trainer with the generation callback
    trainer = pl.Trainer(default_root_dir=os.path.join(CHECKPOINT_PATH, f"cifar10_{latent_dim}"),
                         gpus=1 if str(device).startswith("cuda") else 0,
                         max_epochs=500,
                         callbacks=[ModelCheckpoint(save_weights_only=True),
                                    GenerateCallback(get_train_images(8), every_n_epochs=10),
                                    LearningRateMonitor("epoch")])
    trainer.logger._log_graph = True         # If True, we plot the computation graph in tensorboard
    trainer.logger._default_hp_metric = None # Optional logging argument that we don't need

    # Check whether pretrained model exists. If yes, load it and skip training
    pretrained_filename = os.path.join(CHECKPOINT_PATH, f"cifar10_{latent_dim}.ckpt")
    if os.path.isfile(pretrained_filename):
        print("Found pretrained model, loading...")
        model = Autoencoder.load_from_checkpoint(pretrained_filename)
    else:
        model = Autoencoder(base_channel_size=32, latent_dim=latent_dim)
        trainer.fit(model, train_loader, val_loader)
    # Test best model on validation and test set
    val_result = trainer.test(model, val_loader, verbose=False)
    test_result = trainer.test(model, test_loader, verbose=False)
    result = {"test": test_result, "val": val_result}
    return model, result

In [ ]:
# define any number of nn.Modules (or use your current ones)
encoder = nn.Sequential(nn.Linear(128 * 128 * 3, 64), nn.ReLU(), nn.Linear(64, 3))
decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 128 * 128 * 3))

In [ ]:
# define the LightningModule
class LitAutoEncoder(pl.LightningModule):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        x, y = batch
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        x_hat = self.decoder(z)
        loss = nn.functional.mse_loss(x_hat, x)
        # Logging to TensorBoard by default
        self.log("train_loss", loss)
        return loss

    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), lr=1e-3)
        return optimizer

In [ ]:
# init the autoencoder
autoencoder = LitAutoEncoder(encoder, decoder)

In [ ]:
dataloader

In [ ]:
# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=1)

In [ ]:
trainer.fit(model=autoencoder, train_dataloaders=dataloader)

In [ ]:
trainer

In [ ]:
# load checkpoint
checkpoint = "./lightning_logs/version_0/checkpoints/epoch=0-step=100.ckpt"
autoencoder = LitAutoEncoder.load_from_checkpoint(checkpoint, encoder=encoder, decoder=decoder)

In [ ]:
# choose your trained nn.Module
encoder = autoencoder.encoder
encoder.eval()

In [ ]:
fake_image_batch = Tensor(4, 128 * 128 * 3)
embeddings = encoder(fake_image_batch)
print("⚡" * 20, "\nPredictions (4 image embeddings):\n", embeddings, "\n", "⚡" * 20)

In [ ]:
from torch import randn

In [ ]:
encoder(randn(1, 128*128*3))

In [ ]:
encoder(randn(1, 128*128*3))

In [ ]:
from torch import nn, randn, sigmoid

In [ ]:
randn(1, 128, 128, 3).shape

In [ ]:
my_tensor = randn(32, 3, 128, 128)

In [ ]:
class SimpleEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_size = 32
        self.image_size = [128, 128]
        self.num_channels = 3

        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.image_size[0]*self.image_size[1]*self.num_channels, 128),
            nn.ReLU(),
            nn.Linear(128, self.embedding_size)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return x

In [ ]:
encoder = SimpleEncoder()

In [ ]:
encoder(my_tensor)

In [ ]:
encoder(my_tensor).shape

In [ ]:
encoder

### convolutional

In [ ]:
class SimpleConvEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_size = 32
        self.image_size = [128, 128]
        self.num_channels = 3

        self.conv_encoder = nn.Sequential(
                    nn.Conv2d(3, 32, stride=(1, 1), kernel_size=(5, 5)),
                    nn.LeakyReLU(0.01),
                    nn.Conv2d(32, 64, stride=(2, 2), kernel_size=(3, 3)),
                    nn.LeakyReLU(0.01),
                    nn.Conv2d(64, 64, stride=(2, 2), kernel_size=(3, 3)),
                    nn.LeakyReLU(0.01),
                    nn.Conv2d(64, 64, stride=(1, 1), kernel_size=(3, 3))
            )
        
        self.flatten = nn.Flatten()
        self.lin_encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(50176, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.Linear(128, self.embedding_size)
        )
    
    def forward(self, x):
        BS = x.shape[0]
        x = x.view(BS, 3, self.image_size[0], self.image_size[1])
        x = self.conv_encoder(x)
        x = self.flatten(x)
        x = self.lin_encoder(x)
        return x

In [ ]:
encoder = SimpleConvEncoder()

In [ ]:
encoder(my_tensor)

In [ ]:
encoder(my_tensor).shape

In [ ]:
encoder

In [ ]:
class DecoderConvBase(nn.Module):
    def __init__(self):
        super().__init__()
        self.image_size = [128, 128]
        self.embedding_size = 32

        self.decoder_lin = nn.Sequential(
            nn.Linear(self.embedding_size, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 262144),
            nn.ReLU()
        )

        self.unflatten = nn.Unflatten(
            dim=1, 
            unflattened_size=(256, 32, 32)
        )

        self.decoder_conv = nn.Sequential(
            nn.ConvTranspose2d(256, 64, 2, stride=2, padding=0),
            nn.LeakyReLU(0.01),
            nn.ConvTranspose2d(64, 3, 2, stride=2)
        )
        self.sigmoid = sigmoid

    def forward(self, x):
        x = self.decoder_lin(x)
        x = self.unflatten(x)
        x = self.decoder_conv(x)
        x = self.sigmoid(x)
        return x

In [ ]:
decoder = DecoderConvBase()

In [ ]:
decoder

In [ ]:
decoder(encoder(my_tensor))

In [ ]:
decoder(encoder(my_tensor)).shape